# **Import Libraries**

In [40]:

import pandas as pd 
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
from nltk.stem.isri import ISRIStemmer
stemmer = ISRIStemmer()
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from gensim.models import Word2Vec



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Read Data**

In [27]:
data=pd.read_csv(r"C:\Users\Dell\Downloads\arabic_categorization_data.csv\arabic_categorization_data.csv")
data.head()

,Unnamed: 0,text,type
0,0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture
1,1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture
2,2,\nاستنكرت إدارة المسرح الوطني التونسي الحملة ا...,culture
3,3,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture
4,4,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10366 entries, 0 to 10365
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10366 non-null  int64 
 1   text        10366 non-null  object
 2   type        10366 non-null  object
dtypes: int64(1), object(2)
memory usage: 243.1+ KB


In [29]:
data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
data.head()

,text,type
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture
2,\nاستنكرت إدارة المسرح الوطني التونسي الحملة ا...,culture
3,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture
4,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture


# **Preprocessing**

In [30]:
def clean_text(text):
    # Remove diacritics (tashkeel)
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [31]:
def normalize_text(text):
    # Normalize Arabic characters
    text = text.replace('أ', 'ا').replace('إ', 'ا').replace('آ', 'ا')
    text = text.replace('ى', 'ي').replace('ة', 'ه')
    return text

In [32]:
def tokenize_text(text):
    # Split text into tokens based on whitespace
    tokens = text.split()
    return tokens

In [33]:
arabic_stop_words = set(stopwords.words('arabic'))
def remove_stopwords(tokens):
    # Remove stopwords from the token list
    filtered_tokens = [token for token in tokens if token not in arabic_stop_words]
    return filtered_tokens

In [34]:
def stem_tokens(tokens):
    # Apply ISRIStemmer to each token
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return stemmed_tokens

In [35]:
def segment_sentences(text):
    # Split text into sentences using Arabic punctuation (۔, !, ؟)
    sentence_pattern = r'[۔!؟]+'
    sentences = [s.strip() for s in re.split(sentence_pattern, text) if s.strip()]
    return sentences

# **Apply all the Functions**

In [36]:
def preprocess_pipeline(text):
    # Apply each preprocessing step sequentially
    cleaned_text = clean_text(text)
    normalized_text = normalize_text(cleaned_text)
    tokens = tokenize_text(normalized_text)
    filtered_tokens = remove_stopwords(tokens)
    stemmed_tokens = stem_tokens(filtered_tokens)
    processed_text = ' '.join(stemmed_tokens)
    # Segment sentences from the cleaned text (before tokenization)
    sentences = segment_sentences(cleaned_text)
    return processed_text, sentences

In [37]:
data['processed_text'], data['sentences'] = zip(*data['text'].apply(preprocess_pipeline))

In [38]:
data.head()

,text,type,processed_text,sentences
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture,شرف رئس جمهور بجي قيد سبس اليوم قصر قرطاج علي ...,[أشرف رئيس الجمهورية الباجي قايد السبسي اليوم ...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture,حصل كتب صحف وقراءاته الف بحث ونس تخصص علي جئز ...,[تحصل كتاب المصحف وقراءاته الذي ألفه باحثون تو...
2,\nاستنكرت إدارة المسرح الوطني التونسي الحملة ا...,culture,نكر دره سرح وطن ونس حمل شنه رلم كوت ضذ سرح وطن...,[استنكرت إدارة المسرح الوطني التونسي الحملة ال...
3,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture,حضن جنح ونس قره دول فلم بمد رنس تضف دور سبع هر...,[احتضن جناح تونس في القرية الدولية للأفلام بمد...
4,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture,شهد برل امس جمع فتح سجد فرد نوع علي اقل علي ست...,[شهدت برلين أمس الجمعة افتتاح مسجد فريد من نوع...


In [39]:
data.drop(columns=['text','sentences'],axis=1,inplace=True)
data.head()

,type,processed_text
0,culture,شرف رئس جمهور بجي قيد سبس اليوم قصر قرطاج علي ...
1,culture,حصل كتب صحف وقراءاته الف بحث ونس تخصص علي جئز ...
2,culture,نكر دره سرح وطن ونس حمل شنه رلم كوت ضذ سرح وطن...
3,culture,حضن جنح ونس قره دول فلم بمد رنس تضف دور سبع هر...
4,culture,شهد برل امس جمع فتح سجد فرد نوع علي اقل علي ست...


# **embedding**

In [41]:
# Prepare tokenized sentences for Word2Vec
tokenized_texts = [text.split() for text in data['processed_text']]

# Train Word2Vec model
word2vec_model = Word2Vec(
    sentences=tokenized_texts,
    vector_size=100,  # Same as embedding_dim
    window=5,
    min_count=1,
    workers=4,
    sg=1  # Use Skip-gram (sg=1) or CBOW (sg=0)
)

# Save the Word2Vec model (optional)
word2vec_model.save("word2vec_model.bin")

# **Convert To Sequence**

In [65]:
max_words = 10000  # Maximum number of words to consider in vocabulary
max_len = 100      # Maximum sequence length

# Initialize the tokenizer
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['processed_text'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(data['processed_text'])

# Pad sequences to ensure uniform length
X = pad_sequences(sequences, maxlen=max_len)

# **Split Data**

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['type'])


In [69]:
len(label_encoder.classes_)

9

In [73]:
num_classes=9

In [67]:
y = to_categorical(y, num_classes=num_classes)

In [68]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# **Initialize The Model**

In [70]:
embedding_dim = 100  # Size of the embedding vectors

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(9, activation='softmax'))

c:\Users\Dell\anaconda3\envs\hegiproject\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# **compile The Model**

In [71]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# **Train The Model**

In [72]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=10,
                    batch_size=32,
                    verbose=1)

Epoch 1/10


227/227 ━━━━━━━━━━━━━━━━━━━━ 25s 89ms/step - accuracy: 0.4858 - loss: 1.6695 - val_accuracy: 0.6289 - val_loss: 1.0296
Epoch 2/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 20s 87ms/step - accuracy: 0.6929 - loss: 0.8762 - val_accuracy: 0.6926 - val_loss: 1.0113
Epoch 3/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 20s 89ms/step - accuracy: 0.7734 - loss: 0.6324 - val_accuracy: 0.6990 - val_loss: 0.8713
Epoch 4/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 21s 91ms/step - accuracy: 0.8315 - loss: 0.4950 - val_accuracy: 0.7048 - val_loss: 0.9342
Epoch 5/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 22s 96ms/step - accuracy: 0.8587 - loss: 0.4004 - val_accuracy: 0.7177 - val_loss: 0.8987
Epoch 6/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 24s 107ms/step - accuracy: 0.8940 - loss: 0.3089 - val_accuracy: 0.7035 - val_loss: 0.9283
Epoch 7/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 25s 109ms/step - accuracy: 0.9149 - loss: 0.2533 - val_accuracy: 0.7344 - val_loss: 1.0270
Epoch 8/10
227/227 ━━━━━━━━━━━━━━━━━━━━ 28s 123ms/step - accuracy: 0.9369 - loss: 0.1838 - val_accu

In [74]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 100, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,378,269 (12.89 MB)

 Trainable params: 1,126,089 (4.30 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,252,180 (8.59 MB)